In [1]:
import duckdb
import os

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [ ]:
# %%sql
# PRAGMA temp_directory='C:/Eugenio/Maestria/DMEyF/temp/';
# SET max_temp_directory_size = '20GB'; # by default it's 90% of the free space in your disk

Elimino las variables que tienen data drifting / concept drifting

### Read data

In [2]:
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'
dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_02.parquet'

In [3]:
%%sql

create or replace table competencia_02 as
select *
from read_parquet("{{dataset_path + dataset_file}}");

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [4]:
# para poder ver el tipo de dato de cada columna
df_columns = %sql SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'competencia_02';

df_columns.loc[df_columns['data_type'] == "VARCHAR"]

,column_name,data_type
108,tmobile_app,VARCHAR
109,cmobile_app_trx,VARCHAR
154,clase_ternaria,VARCHAR


In [5]:
%%sql # corregir los tipos mal asignados

ALTER TABLE competencia_02
ALTER COLUMN cmobile_app_trx TYPE DOUBLE;

ALTER TABLE competencia_02
ALTER COLUMN tmobile_app TYPE DOUBLE;

,Success


In [6]:
%%sql
PIVOT competencia_02
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,201901,688,718,123346
1,201902,720,688,124391
2,201903,688,760,124988
3,201904,759,579,125658
4,201905,580,660,126419
5,201906,662,608,127916
6,201907,609,689,129426
7,201908,683,552,131429
8,201909,553,576,133185
9,201910,583,624,135475


In [7]:
%%sql 

select * from competencia_02

limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249677635,201906,1,0,1,60,245,447.14,5952.77,239.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONTINUA
1,249677635,201907,1,0,1,60,246,136.78,5470.83,292.06,...,0.0,0.0,-6752.17,0.0,4350.0,4010.24,8.0,0.0,656.88,CONTINUA
2,249677635,201908,1,0,1,60,247,1738.70,6741.77,1966.29,...,3.0,0.0,-9071.27,0.0,4381.0,4244.58,8.0,0.0,656.88,CONTINUA
3,249677635,201909,1,0,1,61,248,62.21,6329.29,74.07,...,5.0,0.0,-9145.56,0.0,4411.0,4434.52,7.0,0.0,645.15,CONTINUA
4,249677635,201910,1,0,1,61,249,0.00,0.00,0.00,...,1.0,0.0,-8904.51,0.0,4442.0,3826.94,7.0,0.0,504.39,CONTINUA
5,249677635,201911,1,0,1,61,250,242.15,5950.06,529.15,...,3.0,0.0,-6917.83,0.0,4472.0,6895.22,9.0,0.0,727.26,CONTINUA
6,249677635,201912,1,0,1,61,251,394.98,6132.69,740.70,...,6.0,0.0,-11165.90,0.0,4503.0,6546.38,11.0,0.0,903.21,CONTINUA
7,249677635,202001,1,0,1,61,252,192.97,5848.28,568.41,...,2.0,0.0,-13141.00,0.0,4534.0,5745.44,7.0,0.0,809.37,CONTINUA
8,249677635,202002,1,0,1,61,253,256.13,5529.05,391.82,...,3.0,0.0,-12289.49,0.0,4563.0,5571.42,6.0,0.0,738.99,CONTINUA
9,249677635,202003,1,0,1,61,254,642.80,5650.85,1386.04,...,6.0,0.0,-10838.50,0.0,4594.0,6819.69,6.0,0.0,1442.79,CONTINUA


### Elimino variables con data drifting / concept drifting

In [8]:
%%sql
create or replace table competencia_02 as
select * exclude(cprestamos_personales,mprestamos_personales)
from competencia_02

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


### Sumas de columnas

In [9]:
%%sql
create or replace table competencia_02 as
select
    *
    , mcuenta_corriente_adicional + mcuenta_corriente + mcaja_ahorro_adicional + mcaja_ahorro_dolares + mcaja_ahorro AS suma_alt # ninguna tiene null
    , cseguro_vida + cseguro_auto +	cseguro_vivienda + cseguro_accidentes_personales AS cseguro_todos # ninguna tiene null
    , ifnull(matm, 0) + ifnull(matm_other, 0) as matm_total
    
    , ifnull(Visa_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tarjeta_msaldototal
    , ifnull(mactivos_margen, 0) +  ifnull(mpasivos_margen, 0) as map_margen
    , ifnull(ccuenta_corriente, 0) +  ifnull(ccaja_ahorro, 0) as cc_ca
    , ifnull(ctarjeta_visa, 0) +  ifnull(ctarjeta_Visa, 0) as ctarjeta_total
    , ifnull(ctarjeta_visa_transacciones, 0) +  ifnull(ctarjeta_master_transacciones, 0) as ctarjeta_total_transacciones
    , ifnull(mtarjeta_visa_consumo, 0) +  ifnull(mtarjeta_master_consumo, 0) as mtarjeta_total_consumo
    , ifnull(mplazo_fijo_dolares, 0) +  ifnull(mplazo_fijo_pesos, 0) as mplazo_fijo
    , ifnull(minversion1_pesos, 0) +  ifnull(minversion1_dolares, 0) as minversion1
    , ifnull(mpayroll, 0) +  ifnull(mpayroll2, 0) as mpayroll_todos
    , ifnull(mcuenta_debitos_automaticos, 0) +  ifnull(mtarjeta_total_consumo, 0) as consumos_mas_DA
    , ifnull(ctarjeta_visa_debitos_automaticos, 0) +  ifnull(ctarjeta_master_debitos_automaticos, 0) as ctarjeta_todas_debitos_automaticos
    , ifnull(mttarjeta_visa_debitos_automaticos, 0) +  ifnull(mttarjeta_master_debitos_automaticos, 0) as mtarjeta_todas_debitos_automaticos
    , ifnull(ctarjeta_visa_descuentos, 0) +  ifnull(ctarjeta_master_descuentos, 0) as ctarjeta_todas_descuentos
    , ifnull(mtarjeta_visa_descuentos, 0) +  ifnull(mtarjeta_master_descuentos, 0) as mtarjeta_todas_descuentos
    , ifnull(ccomisiones_mantenimiento, 0) +  ifnull(ccomisiones_otras, 0) as comisiones
    , ifnull(mcomisiones_mantenimiento, 0) +  ifnull(mcomisiones_otras, 0) as mcomisiones_suma
    , ifnull(ctransferencias_recibidas, 0) +  ifnull(ctransferencias_emitidas, 0) as ctransferencias
    , ifnull(mtransferencias_recibidas, 0) +  ifnull(mtransferencias_emitidas, 0) as mtransferencias
    , ifnull(Master_msaldopesos, 0) +  ifnull(Visa_msaldopesos, 0) as tarjetas_msaldopesos
    , ifnull(Master_msaldodolares, 0) +  ifnull(Visa_msaldodolares, 0) as tarjetas_msaldodolares
    , ifnull(Master_mconsumospesos, 0) +  ifnull(Visa_mconsumospesos, 0) as tarjetas_mconsumospesos
    , ifnull(Master_mconsumosdolares, 0) +  ifnull(Visa_mconsumosdolares, 0) as tarjetas_mconsumosdolares
    , ifnull(Master_mlimitecompra, 0) +  ifnull(Visa_mlimitecompra, 0) as tarjetas_mlimitecompra
    , ifnull(Master_madelantopesos, 0) +  ifnull(Visa_madelantopesos, 0) as tarjetas_madelantopesos
    , ifnull(Master_madelantodolares, 0) +  ifnull(Visa_madelantodolares, 0) as tarjetas_madelantodolares
    , ifnull(Master_mpagado, 0) +  ifnull(Visa_mpagado, 0) as tarjetas_mpagado
    , ifnull(Master_mpagospesos, 0) +  ifnull(Visa_mpagospesos, 0) as tarjetas_mpagospesos
    , ifnull(Master_mpagosdolares, 0) +  ifnull(Visa_mpagosdolares, 0) as tarjetas_mpagosdolares
    , ifnull(Master_mconsumototal, 0) +  ifnull(Visa_mconsumototal, 0) as tarjetas_mconsumototal
    , ifnull(Master_cconsumos, 0) +  ifnull(Visa_cconsumos, 0) as tarjetas_cconsumos
    , ifnull(Master_cadelantosefectivo, 0) +  ifnull(Visa_cadelantosefectivo, 0) as tarjetas_cadelantosefectivo
    , ifnull(Master_mpagominimo, 0) +  ifnull(Visa_mpagominimo, 0) as tarjetas_mpagominimo
    , ifnull(Master_mfinanciacion_limite, 0) +  ifnull(Visa_mfinanciacion_limite, 0) as tarjetas_mfinanciacion_limite
from competencia_02

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


### Promedios moviles

Variables originales

In [10]:
%%sql
create or replace table competencia_02 as
select
    *
    , avg(mrentabilidad) over ventana_3 as avg3_mrentabilidad
    , avg(mrentabilidad_annual) over ventana_3 as avg3_mrentabilidad_annual
    , avg(mcomisiones) over ventana_3 as avg3_mcomisiones
    , avg(mactivos_margen) over ventana_3 as avg3_mactivos_margen
    , avg(mpasivos_margen) over ventana_3 as avg3_mpasivos_margen
    , avg(cproductos) over ventana_3 as avg3_cproductos
    , avg(tcuentas) over ventana_3 as avg3_tcuentas
    , avg(ccuenta_corriente) over ventana_3 as avg3_ccuenta_corriente
    , avg(mcuenta_corriente_adicional) over ventana_3 as avg3_mcuenta_corriente_adicional
    , avg(mcuenta_corriente) over ventana_3 as avg3_mcuenta_corriente
    , avg(ccaja_ahorro) over ventana_3 as avg3_ccaja_ahorro
    , avg(mcaja_ahorro) over ventana_3 as avg3_mcaja_ahorro
    , avg(mcaja_ahorro_adicional) over ventana_3 as avg3_mcaja_ahorro_adicional
    , avg(mcaja_ahorro_dolares) over ventana_3 as avg3_mcaja_ahorro_dolares
    , avg(mcuentas_saldo) over ventana_3 as avg3_mcuentas_saldo
    , avg(ctarjeta_debito) over ventana_3 as avg3_ctarjeta_debito
    , avg(ctarjeta_debito_transacciones) over ventana_3 as avg3_ctarjeta_debito_transacciones
    , avg(mautoservicio) over ventana_3 as avg3_mautoservicio
    , avg(ctarjeta_visa) over ventana_3 as avg3_ctarjeta_visa
    , avg(ctarjeta_visa_transacciones) over ventana_3 as avg3_ctarjeta_visa_transacciones
    , avg(mtarjeta_visa_consumo) over ventana_3 as avg3_mtarjeta_visa_consumo
    , avg(ctarjeta_master) over ventana_3 as avg3_ctarjeta_master
    , avg(ctarjeta_master_transacciones) over ventana_3 as avg3_ctarjeta_master_transacciones
    , avg(mtarjeta_master_consumo) over ventana_3 as avg3_mtarjeta_master_consumo
    # , avg(cprestamos_personales) over ventana_3 as avg3_cprestamos_personales
    # , avg(mprestamos_personales) over ventana_3 as avg3_mprestamos_personales
    , avg(cprestamos_prendarios) over ventana_3 as avg3_cprestamos_prendarios
    , avg(mprestamos_prendarios) over ventana_3 as avg3_mprestamos_prendarios
    , avg(cprestamos_hipotecarios) over ventana_3 as avg3_cprestamos_hipotecarios
    , avg(mprestamos_hipotecarios) over ventana_3 as avg3_mprestamos_hipotecarios
    , avg(cplazo_fijo) over ventana_3 as avg3_cplazo_fijo
    , avg(mplazo_fijo_dolares) over ventana_3 as avg3_mplazo_fijo_dolares
    , avg(mplazo_fijo_pesos) over ventana_3 as avg3_mplazo_fijo_pesos
    , avg(cinversion1) over ventana_3 as avg3_cinversion1
    , avg(minversion1_pesos) over ventana_3 as avg3_minversion1_pesos
    , avg(minversion1_dolares) over ventana_3 as avg3_minversion1_dolares
    , avg(cinversion2) over ventana_3 as avg3_cinversion2
    , avg(minversion2) over ventana_3 as avg3_minversion2
    , avg(cseguro_vida) over ventana_3 as avg3_cseguro_vida
    , avg(cseguro_auto) over ventana_3 as avg3_cseguro_auto
    , avg(cseguro_vivienda) over ventana_3 as avg3_cseguro_vivienda
    , avg(cseguro_accidentes_personales) over ventana_3 as avg3_cseguro_accidentes_personales
    , avg(cpayroll_trx) over ventana_3 as avg3_cpayroll_trx
    , avg(mpayroll) over ventana_3 as avg3_mpayroll
    , avg(mpayroll2) over ventana_3 as avg3_mpayroll2
    , avg(cpayroll2_trx) over ventana_3 as avg3_cpayroll2_trx
    , avg(ccuenta_debitos_automaticos) over ventana_3 as avg3_ccuenta_debitos_automaticos
    , avg(mcuenta_debitos_automaticos) over ventana_3 as avg3_mcuenta_debitos_automaticos
    , avg(ctarjeta_visa_debitos_automaticos) over ventana_3 as avg3_ctarjeta_visa_debitos_automaticos
    , avg(mttarjeta_visa_debitos_automaticos) over ventana_3 as avg3_mttarjeta_visa_debitos_automaticos
    , avg(ctarjeta_master_debitos_automaticos) over ventana_3 as avg3_ctarjeta_master_debitos_automaticos
    , avg(mttarjeta_master_debitos_automaticos) over ventana_3 as avg3_mttarjeta_master_debitos_automaticos
    , avg(cpagodeservicios) over ventana_3 as avg3_cpagodeservicios
    , avg(mpagodeservicios) over ventana_3 as avg3_mpagodeservicios
    , avg(cpagomiscuentas) over ventana_3 as avg3_cpagomiscuentas
    , avg(mpagomiscuentas) over ventana_3 as avg3_mpagomiscuentas
    , avg(ccajeros_propios_descuentos) over ventana_3 as avg3_ccajeros_propios_descuentos
    , avg(mcajeros_propios_descuentos) over ventana_3 as avg3_mcajeros_propios_descuentos
    , avg(ctarjeta_visa_descuentos) over ventana_3 as avg3_ctarjeta_visa_descuentos
    , avg(mtarjeta_visa_descuentos) over ventana_3 as avg3_mtarjeta_visa_descuentos
    , avg(ctarjeta_master_descuentos) over ventana_3 as avg3_ctarjeta_master_descuentos
    , avg(mtarjeta_master_descuentos) over ventana_3 as avg3_mtarjeta_master_descuentos
    , avg(ccomisiones_mantenimiento) over ventana_3 as avg3_ccomisiones_mantenimiento
    , avg(mcomisiones_mantenimiento) over ventana_3 as avg3_mcomisiones_mantenimiento
    , avg(ccomisiones_otras) over ventana_3 as avg3_ccomisiones_otras
    , avg(mcomisiones_otras) over ventana_3 as avg3_mcomisiones_otras
    , avg(cforex) over ventana_3 as avg3_cforex
    , avg(cforex_buy) over ventana_3 as avg3_cforex_buy
    , avg(mforex_buy) over ventana_3 as avg3_mforex_buy
    , avg(cforex_sell) over ventana_3 as avg3_cforex_sell
    , avg(mforex_sell) over ventana_3 as avg3_mforex_sell
    , avg(ctransferencias_recibidas) over ventana_3 as avg3_ctransferencias_recibidas
    , avg(mtransferencias_recibidas) over ventana_3 as avg3_mtransferencias_recibidas
    , avg(ctransferencias_emitidas) over ventana_3 as avg3_ctransferencias_emitidas
    , avg(mtransferencias_emitidas) over ventana_3 as avg3_mtransferencias_emitidas
    , avg(cextraccion_autoservicio) over ventana_3 as avg3_cextraccion_autoservicio
    , avg(mextraccion_autoservicio) over ventana_3 as avg3_mextraccion_autoservicio
    , avg(ccheques_depositados) over ventana_3 as avg3_ccheques_depositados
    , avg(mcheques_depositados) over ventana_3 as avg3_mcheques_depositados
    , avg(ccheques_emitidos) over ventana_3 as avg3_ccheques_emitidos
    , avg(mcheques_emitidos) over ventana_3 as avg3_mcheques_emitidos
    , avg(ccheques_depositados_rechazados) over ventana_3 as avg3_ccheques_depositados_rechazados
    , avg(mcheques_depositados_rechazados) over ventana_3 as avg3_mcheques_depositados_rechazados
    , avg(ccheques_emitidos_rechazados) over ventana_3 as avg3_ccheques_emitidos_rechazados
    , avg(mcheques_emitidos_rechazados) over ventana_3 as avg3_mcheques_emitidos_rechazados
    , avg(ccallcenter_transacciones) over ventana_3 as avg3_ccallcenter_transacciones
    , avg(chomebanking_transacciones) over ventana_3 as avg3_chomebanking_transacciones
    , avg(ccajas_transacciones) over ventana_3 as avg3_ccajas_transacciones
    , avg(ccajas_consultas) over ventana_3 as avg3_ccajas_consultas
    , avg(ccajas_depositos) over ventana_3 as avg3_ccajas_depositos
    , avg(ccajas_extracciones) over ventana_3 as avg3_ccajas_extracciones
    , avg(ccajas_otras) over ventana_3 as avg3_ccajas_otras
    , avg(catm_trx) over ventana_3 as avg3_catm_trx
    , avg(matm) over ventana_3 as avg3_matm
    , avg(catm_trx_other) over ventana_3 as avg3_catm_trx_other
    , avg(matm_other) over ventana_3 as avg3_matm_other
    , avg(ctrx_quarter) over ventana_3 as avg3_ctrx_quarter
    , avg(cmobile_app_trx) over ventana_3 as avg3_cmobile_app_trx
    , avg(Master_mfinanciacion_limite) over ventana_3 as avg3_Master_mfinanciacion_limite
    , avg(Master_msaldototal) over ventana_3 as avg3_Master_msaldototal
    , avg(Master_msaldopesos) over ventana_3 as avg3_Master_msaldopesos
    , avg(Master_msaldodolares) over ventana_3 as avg3_Master_msaldodolares
    , avg(Master_mconsumospesos) over ventana_3 as avg3_Master_mconsumospesos
    , avg(Master_mconsumosdolares) over ventana_3 as avg3_Master_mconsumosdolares
    , avg(Master_mlimitecompra) over ventana_3 as avg3_Master_mlimitecompra
    , avg(Master_madelantopesos) over ventana_3 as avg3_Master_madelantopesos
    , avg(Master_madelantodolares) over ventana_3 as avg3_Master_madelantodolares
    , avg(Master_mpagado) over ventana_3 as avg3_Master_mpagado
    , avg(Master_mpagospesos) over ventana_3 as avg3_Master_mpagospesos
    , avg(Master_mpagosdolares) over ventana_3 as avg3_Master_mpagosdolares
    , avg(Master_mconsumototal) over ventana_3 as avg3_Master_mconsumototal
    , avg(Master_cconsumos) over ventana_3 as avg3_Master_cconsumos
    , avg(Master_cadelantosefectivo) over ventana_3 as avg3_Master_cadelantosefectivo
    , avg(Master_mpagominimo) over ventana_3 as avg3_Master_mpagominimo
    , avg(Visa_mfinanciacion_limite) over ventana_3 as avg3_Visa_mfinanciacion_limite
    , avg(Visa_msaldototal) over ventana_3 as avg3_Visa_msaldototal
    , avg(Visa_msaldopesos) over ventana_3 as avg3_Visa_msaldopesos
    , avg(Visa_msaldodolares) over ventana_3 as avg3_Visa_msaldodolares
    , avg(Visa_mconsumospesos) over ventana_3 as avg3_Visa_mconsumospesos
    , avg(Visa_mconsumosdolares) over ventana_3 as avg3_Visa_mconsumosdolares
    , avg(Visa_mlimitecompra) over ventana_3 as avg3_Visa_mlimitecompra
    , avg(Visa_madelantopesos) over ventana_3 as avg3_Visa_madelantopesos
    , avg(Visa_madelantodolares) over ventana_3 as avg3_Visa_madelantodolares
    , avg(Visa_mpagado) over ventana_3 as avg3_Visa_mpagado
    , avg(Visa_mpagospesos) over ventana_3 as avg3_Visa_mpagospesos
    , avg(Visa_mpagosdolares) over ventana_3 as avg3_Visa_mpagosdolares
    , avg(Visa_mconsumototal) over ventana_3 as avg3_Visa_mconsumototal
    , avg(Visa_cconsumos) over ventana_3 as avg3_Visa_cconsumos
    , avg(Visa_cadelantosefectivo) over ventana_3 as avg3_Visa_cadelantosefectivo
    , avg(Visa_mpagominimo) over ventana_3 as avg3_Visa_mpagominimo
from competencia_02
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Variables creadas

In [11]:
%%sql
create or replace table competencia_02 as
select
    *
    , avg(map_margen) over ventana_3 as avg3_map_margen
    , avg(cc_ca) over ventana_3 as avg3_cc_ca
    , avg(ctarjeta_total) over ventana_3 as avg3_ctarjeta_total
    , avg(ctarjeta_total_transacciones) over ventana_3 as avg3_ctarjeta_total_transacciones
    , avg(mtarjeta_total_consumo) over ventana_3 as avg3_mtarjeta_total_consumo
    , avg(mplazo_fijo) over ventana_3 as avg3_mplazo_fijo
    , avg(minversion1) over ventana_3 as avg3_minversion1
    , avg(cseguro_todos) over ventana_3 as avg3_cseguro_todos
    , avg(mpayroll_todos) over ventana_3 as avg3_mpayroll_todos
    , avg(consumos_mas_DA) over ventana_3 as avg3_consumos_mas_DA
    , avg(ctarjeta_todas_debitos_automaticos) over ventana_3 as avg3_ctarjeta_todas_debitos_automaticos
    , avg(mtarjeta_todas_debitos_automaticos) over ventana_3 as avg3_mtarjeta_todas_debitos_automaticos
    , avg(ctarjeta_todas_descuentos) over ventana_3 as avg3_ctarjeta_todas_descuentos
    , avg(mtarjeta_todas_descuentos) over ventana_3 as avg3_mtarjeta_todas_descuentos
    , avg(comisiones) over ventana_3 as avg3_comisiones
    , avg(mcomisiones_suma) over ventana_3 as avg3_mcomisiones_suma
    , avg(ctransferencias) over ventana_3 as avg3_ctransferencias
    , avg(mtransferencias) over ventana_3 as avg3_mtransferencias
    , avg(tarjeta_msaldototal) over ventana_3 as avg3_tarjeta_msaldototal
    , avg(tarjetas_msaldopesos) over ventana_3 as avg3_tarjetas_msaldopesos
    , avg(tarjetas_msaldodolares) over ventana_3 as avg3_tarjetas_msaldodolares
    , avg(tarjetas_mconsumospesos) over ventana_3 as avg3_tarjetas_mconsumospesos
    , avg(tarjetas_mconsumosdolares) over ventana_3 as avg3_tarjetas_mconsumosdolares
    , avg(tarjetas_mlimitecompra) over ventana_3 as avg3_tarjetas_mlimitecompra
    , avg(tarjetas_madelantopesos) over ventana_3 as avg3_tarjetas_madelantopesos
    , avg(tarjetas_madelantodolares) over ventana_3 as avg3_tarjetas_madelantodolares
    , avg(tarjetas_mpagado) over ventana_3 as avg3_tarjetas_mpagado
    , avg(tarjetas_mpagospesos) over ventana_3 as avg3_tarjetas_mpagospesos
    , avg(tarjetas_mpagosdolares) over ventana_3 as avg3_tarjetas_mpagosdolares
    , avg(tarjetas_mconsumototal) over ventana_3 as avg3_tarjetas_mconsumototal
    , avg(tarjetas_cconsumos) over ventana_3 as avg3_tarjetas_cconsumos
    , avg(tarjetas_cadelantosefectivo) over ventana_3 as avg3_tarjetas_cadelantosefectivo
    , avg(tarjetas_mpagominimo) over ventana_3 as avg3_tarjetas_mpagominimo
    , avg(tarjetas_mfinanciacion_limite) over ventana_3 as avg3_tarjetas_mfinanciacion_limite
    , avg(matm_total) over ventana_3 as avg3_matm_total
from competencia_02
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


### Dimensiones

In [12]:
dataset_path

'C:/Eugenio/Maestria/DMEyF/datasets/'

### Guardar datos

In [13]:
# %sql COPY competencia_02 TO 'C:/Eugenio/Maestria/DMEyF/datasets/competencia_02_fe_v01.csv' (FORMAT CSV, HEADER TRUE);
%sql COPY competencia_02 TO '{{dataset_path}}competencia_02_fe_sin_adj.parquet' (FORMAT PARQUET)

# if you want to use dataset_path you need to build the path in another line because once you use %sql or %%sql you can't use a parameter
# outside the function.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
